In [1]:
# import some shit
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import xdeepfm
from deepctr.inputs import SparseFeat, DenseFeat, get_fixlen_feature_names

In [2]:
DEFAULT_HEADER = (
    ["label"]
    + ['I' + str(i) for i in range(1, 14)]
    + ['C' + str(i) for i in range(1, 27)]
)
data = pd.read_csv('./data.txt', sep='\t', header=None, names=DEFAULT_HEADER)
pd.set_option('display.max_columns', None)
data.tail()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
99995,0,1.0,60,37.0,0.0,1.0,0.0,4.0,0.0,23.0,1.0,3.0,NaN,0.0,68fd1e64,91381efc,eacb18f6,f00be896,25c83c98,fe6b92e5,2489e185,985e3fcb,a73ee510,3b08e48b,4d1f7d97,d6b4fe71,954029f8,1adce6ef,a0602981,2a08cb76,e5ba7672,38b82d9f,21ddcdc9,b1252a9d,24e5131b,NaN,32c7478e,4fa16304,47907db5,72966777
99996,1,NaN,0,12.0,NaN,173121.0,NaN,0.0,3.0,10.0,NaN,0.0,NaN,NaN,68fd1e64,f0cf0024,6f67f7e5,41274cd7,4cf72387,fbad5c96,9e5d3694,0b153874,7cc72ec2,be289b53,699034a0,623049e6,a0edec24,1adce6ef,55dc357b,c92f3b61,e5ba7672,b04e4670,21ddcdc9,b1252a9d,60f6221e,NaN,32c7478e,43f13e8b,ea9a246c,731c3655
99997,0,10.0,2,1.0,26.0,482.0,60.0,10.0,11.0,60.0,1.0,1.0,0.0,59.0,8cf07265,c8687797,b063fe4e,4b972461,25c83c98,7e0ccccf,176dc88e,d7c4a8f5,a73ee510,3b08e48b,d2b7c44b,8cdc4941,68637c0d,b28479f6,dc96c4b0,3084c78b,07c540c4,a7e06874,21ddcdc9,a458ea53,514b7308,NaN,32c7478e,2fd70e1c,010f6491,ec26ad35
99998,0,NaN,390,43.0,4.0,345365.0,NaN,0.0,4.0,4.0,NaN,0.0,NaN,4.0,0e78bd46,a796837e,dffca8ba,0fa0d423,25c83c98,fe6b92e5,bc324536,51d76abe,7cc72ec2,5cad6330,2bcfb78f,93bab460,e6fc496d,cfef1c29,f0bf9094,6bb29970,e5ba7672,1cdbd1c5,NaN,NaN,d9d9202f,NaN,32c7478e,8fc66e78,NaN,NaN
99999,0,0.0,-1,137.0,19.0,9504.0,NaN,0.0,22.0,22.0,0.0,0.0,NaN,22.0,05db9164,38d50e09,dd17c91c,82a61820,25c83c98,7e0ccccf,a088b320,5b392875,a73ee510,323dec10,6be5122d,75529ad8,c8c105ca,07d13a8f,ee569ce2,5eea53aa,e5ba7672,582152eb,21ddcdc9,5840adea,0f78ab39,c9d4222a,32c7478e,cafb4e4d,001f3601,99f4f64c


In [3]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]
target = ['label']

In [4]:
data[sparse_features] = data[sparse_features].fillna('-1')
data[dense_features] = data[dense_features].fillna(0)
data[sparse_features]

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,-1,3a171ecb,c5c50484,e8b83407,9727dd16
1,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,-1,3a171ecb,43f13e8b,e8b83407,731c3655
2,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,-1,-1,e587c466,ad3062eb,3a171ecb,3b183c5c,-1,-1
3,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,-1,-1,6b3a5ca6,-1,3a171ecb,9117a34a,-1,-1
4,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,e5d8af57,66a76a26,f06c53ac,1adce6ef,8ff4b403,01adbab4,1e88c74f,26b3c7a7,-1,-1,21c9516a,-1,32c7478e,b34f3128,-1,-1
5,05db9164,6c9c9cf3,2730ec9c,5400db8b,43b19349,6f6d9be8,53b5f978,0b153874,a73ee510,3b08e48b,91e8fc27,be45b877,9ff13f22,07d13a8f,06969a20,9bc7fff5,776ce399,92555263,-1,-1,242bb710,8ec974f4,be7c41b4,72c78f11,-1,-1
6,439a44a4,ad4527a2,c02372d0,d34ebbaa,43b19349,fe6b92e5,4bc6ffea,0b153874,a73ee510,3b08e48b,a4609aab,14d63538,772a00d7,07d13a8f,f9d1382e,b00d3dc9,776ce399,cdfa8259,-1,-1,20062612,-1,93bad2c0,1b256e61,-1,-1
7,68fd1e64,2c16a946,503b9dbc,e4dbea90,f3474129,13718bbd,38eb9cf4,1f89b562,a73ee510,547c0ffe,bc8c9f21,60ab2f07,46f42a63,07d13a8f,18231224,e6b6bdc7,e5ba7672,74ef3502,-1,-1,5316a17f,-1,32c7478e,9117a34a,-1,-1
8,05db9164,d833535f,d032c263,c18be181,25c83c98,7e0ccccf,d5b6acf2,0b153874,a73ee510,2acdcf4e,086ac2d2,dfbb09fb,41a6ae00,b28479f6,e2502ec9,84898b2a,e5ba7672,42a2edb9,-1,-1,0014c32a,-1,32c7478e,3b183c5c,-1,-1
9,05db9164,510b40a5,d03e7c24,eb1fd928,25c83c98,-1,52283d1c,0b153874,a73ee510,015ac893,e51ddf94,951fe4a9,3516f6e6,07d13a8f,2ae4121c,8ec71479,d4bb7bd8,70d0f5f9,-1,-1,0e63fca0,-1,32c7478e,0e8fe315,-1,-1


In [5]:
# Label encoding for sparse features & Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.001799,0.000162,0.000076,0.000000,7.937383e-04,0.000246,0.001703,0.000428,0.014296,0.166667,0.019231,0.000000,0.000305,229,250,43140,12137,23,5,6591,27,2,7231,2691,8957,424,4,2809,18686,9,2422,150,3,1163,0,2,9570,41,5604
1,0,0.003597,0.000108,0.000671,0.002398,5.858271e-05,0.000491,0.000227,0.000428,0.000316,0.166667,0.009615,0.000000,0.000610,229,471,19101,6423,23,11,4407,8,2,1804,1162,15920,2355,17,4700,27298,0,1727,150,1,14512,0,2,3233,41,4215
2,0,0.003597,0.000108,0.000015,0.033573,4.405190e-04,0.005463,0.000454,0.000428,0.019351,0.166667,0.028846,0.006085,0.006862,74,22,489,19188,23,5,5941,8,2,2505,1396,23282,1874,2,2235,7353,6,510,0,0,34644,6,2,2818,0,0
3,0,0.000000,0.048316,0.000000,0.000000,2.522503e-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229,87,29109,4556,23,11,1449,8,2,10355,3420,26353,586,2,482,11287,1,1170,0,0,15997,0,2,6971,0,0
4,0,0.005396,0.000054,0.000000,0.000000,1.148681e-06,0.000000,0.000341,0.000000,0.000000,0.166667,0.009615,0.000000,0.000000,300,348,34237,24514,23,1,5210,8,2,3530,3430,16684,2632,4,2891,237,1,390,0,0,5101,0,1,8663,0,0
5,0,0.000000,0.000054,0.000000,0.000000,7.365340e-03,0.000000,0.000000,0.000000,0.000474,0.000000,0.000000,0.000000,0.000000,8,210,6756,8339,51,4,2588,8,2,2505,2190,30745,1755,2,129,21169,5,1445,0,0,5459,5,10,5503,0,0
6,0,0.000000,0.000162,0.000031,0.000000,1.819510e-03,0.000000,0.000000,0.000214,0.000158,0.000000,0.000000,0.000000,0.000000,135,344,32875,20816,51,11,2342,8,2,2505,2486,3275,1316,2,5111,23922,5,2032,0,0,4823,0,7,1260,0,0
7,1,0.001799,0.000324,0.000031,0.000000,0.000000e+00,0.000000,0.000114,0.000000,0.000000,0.166667,0.009615,0.000000,0.000000,229,87,13772,22531,137,1,1770,27,2,3596,2827,15682,782,2,482,31227,9,1170,0,0,12399,0,1,6971,0,0
8,0,0.000000,0.002483,0.000061,0.019185,1.091821e-02,0.015285,0.003179,0.006628,0.011137,0.000000,0.009615,0.000000,0.001220,8,428,35651,19188,23,5,6352,8,2,1773,117,36115,726,17,4593,18076,9,662,0,0,17,0,1,2818,0,0
9,0,0.000000,0.001997,0.000000,0.002398,1.937652e-02,0.001289,0.000114,0.000428,0.000237,0.000000,0.009615,0.000000,0.000152,8,160,35657,23159,23,0,2534,8,2,62,3420,24080,586,2,889,19456,8,1136,0,0,2169,0,1,681,0,0


In [6]:
# Count unique features for each sparse field & record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features] + [DenseFeat(feat, 1) for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
fixlen_feature_names = get_fixlen_feature_names(linear_feature_columns + dnn_feature_columns)
print(fixlen_feature_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']


In [7]:
# Generate input data for model
train, test = train_test_split(data, test_size=0.2)
train_model_input = [train[name] for name in fixlen_feature_names]
test_model_input = [test[name] for name in fixlen_feature_names]

In [9]:
 # Define Model,train,predict and evaluate
    model = xdeepfm.xDeepFM(linear_feature_columns, dnn_feature_columns)
    model.compile("adam", "binary_crossentropy",
                  metrics=['binary_crossentropy'], )

    history = model.fit(train_model_input, train[target].values,
                        batch_size=64, epochs=1, verbose=1, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=128)
#     print(model.summary())
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

Train on 64000 samples, validate on 16000 samples
Epoch 1/1
64000/64000 [==============================] - 119s 2ms/step - loss: 0.4712 - binary_crossentropy: 0.4686 - val_loss: 0.4651 - val_binary_crossentropy: 0.4593
test LogLoss 0.4546
test AUC 0.7631
